In [1]:
import cv2
import numpy as np
import glob
import os
import math

# =============== [ P A R A M E T E R ] - [ F U L L ]
#=========================================================
pathResultImages = r"Result_Images\\"
BEV_Name = "1.Rectangle-BEV Camera Show (Crop).jpg"
heightBEV = 785
widthBEV = 657
# =============== Camera Filename Images
Camera_A_Name = "Camera_A_1.jpg"
Camera_B_Name = "Camera_B_1.jpg"
Camera_C_Name = "Camera_C_1.jpg"
Camera_D_Name = "Camera_D_1.jpg"
print("=> Filename: ", Camera_A_Name,Camera_B_Name,Camera_C_Name,Camera_D_Name)
print()
# =============== Matrix Camera -> BEV
MatrixA = np.zeros((3,3), np.float)
MatrixB = np.zeros((3,3), np.float)
MatrixC = np.zeros((3,3), np.float)
MatrixD = np.zeros((3,3), np.float)
MatrixA[0:3] = [[ 2.36588339e-01, -9.79616203e-01,  2.31036674e+02], [-9.26623812e-03, -1.35976221e+00,  4.85950598e+02], [-6.85560659e-05, -3.12789910e-03,  1.00000000e+00]]
MatrixB[0:3] = [[-2.19189956e+00,  1.08059785e-01,  4.30714845e+02], [-2.98505403e+00, -4.48153295e-01,  6.36091901e+02], [-7.19621378e-03,  4.49285188e-04,  1.00000000e+00]]
MatrixC[0:3] = [[-1.06122502e+00, -5.80395453e-02,  3.81511526e+02], [-1.29407427e+00,  2.14188277e-01,  3.27761048e+02], [-3.18749314e-03, -1.48513790e-04,  1.00000000e+00]]
MatrixD[0:3] = [[-6.56100043e-01, -2.23582382e+00,  4.97126571e+02], [-4.64843971e-02, -2.77641520e+00,  5.24733577e+02], [-1.21516486e-04, -7.20435663e-03,  1.00000000e+00]]
print("=> MatrixA = ", MatrixA)
print("=> MatrixB = ", MatrixB)
print("=> MatrixC = ", MatrixC)
print("=> MatrixD = ", MatrixD)
print("="*50)
print()
# =============== Coordinate Small Square [BEST] Inside    
pointXMin = 225
pointXMax = 411
pointYMin = 290
pointYMax = 519
print("=> pointXMin = ", pointXMin)
print("=> pointXMax = ", pointXMax)
print("=> pointYMin = ", pointYMin)
print("=> pointYMax = ", pointYMax)
print("="*50)
print()
#=========================================================
# =============== [ P A R A M E T E R ] - [ F U L L ]

# =============== [ F U N C T I O N ] - [ F U L L ]
#=========================================================
def readCamera(cameraName, Camera_Filename):
    img = cv2.imread(Camera_Filename)
    if (cameraName == "D"):
        # Rotate D - No Rotate
        pass
    elif (cameraName == "B"):
        # Camera B - Rotate 90 COUNTERCLOCKWISE
        img = cv2.rotate(img, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
    elif (cameraName == "C"):
        # Camera C - Rotate 90 CLOCKWISE
        img = cv2.rotate(img, cv2.cv2.ROTATE_90_CLOCKWISE)
    elif (cameraName == "A"):
        # Camera A - Rotate 180 
        img = cv2.rotate(img, cv2.cv2.ROTATE_180)
    cv2.imshow(Camera_Filename, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img
#=========================================================
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def showBEV_Result(img,Camera_Filename,matrix, width,height):
    imgBEV = cv2.warpPerspective(img,matrix, (width,height))
    '''
    cv2.imshow("-BEV-" + Camera_Filename, imgBEV)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    '''
    return imgBEV
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def removeSmallSquareFull(cameraName, imgBEVCamera, xMin,xMax,yMin,yMax):
    thickness = -1
    color = (0,0,0)
    if cameraName == "D":
        img = cv2.rectangle(imgBEVCamera, (0,yMin), (imgBEVCamera.shape[1],yMax), color, thickness)
    elif cameraName == "B":
        img = cv2.rectangle(imgBEVCamera, (xMin,0), (xMax,imgBEVCamera.shape[0]), color, thickness)
    elif cameraName == "C":
        img = cv2.rectangle(imgBEVCamera, (xMin,imgBEVCamera.shape[0]), (xMax,0), color, thickness)
    elif cameraName == "A":
        img = cv2.rectangle(imgBEVCamera, (imgBEVCamera.shape[1],yMin), (0,yMax), color, thickness)
    '''
    cv2.imshow("removeSmallSquareFull", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    '''
    return img
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def showLineDividedRemove(cameraName,imgBEVCamera,BEV_Camera_Name, heightBEV,widthBEV):
    color = (0,0,255)
    thickness = 1
    # Divided Into 2 Parts
    halfHeight = int(heightBEV/2)
    halfWidth = int(widthBEV/2)
    if cameraName == "A" or cameraName == "D":
        imgBEVCamera = cv2.line(imgBEVCamera, (0,halfHeight), (widthBEV,halfHeight), color, thickness)
        if cameraName == "D":
            # Removes Buttom Parts
            imgBEVCamera = cv2.rectangle(imgBEVCamera, (0,halfHeight), (widthBEV,halfHeight*2), (0,0,0), -1)
        elif cameraName == "A":
            # Removes Top Parts
            imgBEVCamera = cv2.rectangle(imgBEVCamera, (0,0), (widthBEV,halfHeight), (0,0,0), -1)
    elif cameraName == "C" or cameraName == "B":
        imgBEVCamera = cv2.line(imgBEVCamera, (halfWidth,0), (halfWidth,heightBEV), color, thickness)
        if cameraName == "C":
            # Removes Left Parts
            imgBEVCamera = cv2.rectangle(imgBEVCamera, (0,0), (halfWidth,heightBEV), (0,0,0), -1)
        elif cameraName == "B":
            # Removes Right Parts
            imgBEVCamera = cv2.rectangle(imgBEVCamera, (halfWidth,0), (halfWidth*2,heightBEV), (0,0,0), -1)
    '''
    cv2.imshow("Remove Parts" + BEV_Camera_Name, imgBEVCamera)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    '''
    return imgBEVCamera
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def cropBEVRemovaCamera(cameraName,img,BEVRemove_Camera_Name):
    height = img.shape[0]
    width = img.shape[1]
    if cameraName == "D":
        x = 0
        h = int(height/2)
        y = 0
        w = width
        img = img[y:y+h, x:x+w]
        imgRotate = img
    elif cameraName == "B":
        x = 0
        h = height
        y = 0
        w = int(width/2)
        img = img[y:y+h, x:x+w]
        # Rotate 
        imgRotate = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    elif cameraName == "C":
        x = int(width/2)
        h = height
        y = 0
        w = int(width/2)
        img = img[y:y+h, x:x+w]
        imgRotate = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    elif cameraName == "A":
        x = 0
        h = int(height/2)
        y = int(height/2)
        w = width
        img = img[y:y+h, x:x+w]
        imgRotate = cv2.rotate(img, cv2.ROTATE_180)
    cv2.imshow("-Crop-" + BEVRemove_Camera_Name, img)
    cv2.imshow("-Rotate-" + BEVRemove_Camera_Name, imgRotate)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img, imgRotate
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
#=========================================================
def writeNewImage(img, pathWriteImg):
    cv2.imwrite(pathWriteImg, img)
#=========================================================
# =============== [ F U N C T I O N ] - [ F U L L ]

# =============== [ M A I N - F U N C T I O N ] - [ F U L L ]
#=========================================================
def main():
    print("==> 1. Size BEV")
    print("Height: ", heightBEV)
    print("Width: ", widthBEV)
    print()

    print("==> 2. Read Camera & Rotate")
    print()
    # ==================== Read Camera & Rotate
    imgCameraA = readCamera("A", Camera_A_Name)
    imgCameraB = readCamera("B", Camera_B_Name)
    imgCameraC = readCamera("C", Camera_C_Name)
    imgCameraD = readCamera("D", Camera_D_Name)

    print("==> 2. Show  BEV Result")
    print()
    # Show  BEV Result
    imgCameraA = showBEV_Result(imgCameraA,Camera_A_Name,MatrixA, widthBEV,heightBEV)
    imgCameraB = showBEV_Result(imgCameraB,Camera_B_Name,MatrixB, widthBEV,heightBEV)
    imgCameraC = showBEV_Result(imgCameraC,Camera_C_Name,MatrixC, widthBEV,heightBEV)
    imgCameraD = showBEV_Result(imgCameraD,Camera_D_Name,MatrixD, widthBEV,heightBEV)

    print("==> 3. Remove Small Square [BEST] Inside - [ F U L L ]")
    print()
    # Remove Small Square [BEST] Inside - [ F U L L ]
    imgCameraA = removeSmallSquareFull("A", imgCameraA, pointXMin,pointXMax,pointYMin,pointYMax)
    imgCameraB = removeSmallSquareFull("B", imgCameraB, pointXMin,pointXMax,pointYMin,pointYMax)
    imgCameraC = removeSmallSquareFull("C", imgCameraC, pointXMin,pointXMax,pointYMin,pointYMax)
    imgCameraD = removeSmallSquareFull("D", imgCameraD, pointXMin,pointXMax,pointYMin,pointYMax)

    print("==> 3. Show Line to Divided and Removes")
    print()
    # Show Line to Divided and Removes
    imgCameraA = showLineDividedRemove("A",imgCameraA,Camera_A_Name, heightBEV,widthBEV)
    imgCameraB = showLineDividedRemove("B",imgCameraB,Camera_B_Name, heightBEV,widthBEV)
    imgCameraC = showLineDividedRemove("C",imgCameraC,Camera_C_Name, heightBEV,widthBEV)
    imgCameraD = showLineDividedRemove("D",imgCameraD,Camera_D_Name, heightBEV,widthBEV)

    print("==> 4. Show Line to Divided and Removes")
    print()
    # Read BEV Remove Camera & Crop
    imgCameraA, imgCameraRotateA = cropBEVRemovaCamera("A", imgCameraA,Camera_A_Name)
    imgCameraB, imgCameraRotateB = cropBEVRemovaCamera("B", imgCameraB,Camera_B_Name)
    imgCameraC, imgCameraRotateC = cropBEVRemovaCamera("C", imgCameraC,Camera_C_Name)
    imgCameraD, imgCameraRotateD = cropBEVRemovaCamera("D", imgCameraD,Camera_D_Name)

    print("==> 4. Write new Images - [ F U L L ]")
    print()
    # Write new Images
    filenameImage = "5.1.BEV-FINAL-FULL-"
    filenameImage = pathResultImages + filenameImage
    writeNewImage(imgCameraA, filenameImage+Camera_A_Name)
    writeNewImage(imgCameraB, filenameImage+Camera_B_Name)
    writeNewImage(imgCameraC, filenameImage+Camera_C_Name)
    writeNewImage(imgCameraD, filenameImage+Camera_D_Name)
    # Write new Images Rotate
    filenameImage = "5.2.R.BEV-FINAL-FULL-"
    filenameImage = pathResultImages + filenameImage
    writeNewImage(imgCameraRotateA, filenameImage+Camera_A_Name)
    writeNewImage(imgCameraRotateB, filenameImage+Camera_B_Name)
    writeNewImage(imgCameraRotateC, filenameImage+Camera_C_Name)
    writeNewImage(imgCameraRotateD, filenameImage+Camera_D_Name)
#=========================================================
# =============== [ M A I N - F U N C T I O N ] - [ F U L L ]

if __name__ == '__main__':       
    # Calling main() function 
    main()

=> Filename:  Camera_A_1.jpg Camera_B_1.jpg Camera_C_1.jpg Camera_D_1.jpg

=> MatrixA =  [[ 2.36588339e-01 -9.79616203e-01  2.31036674e+02]
 [-9.26623812e-03 -1.35976221e+00  4.85950598e+02]
 [-6.85560659e-05 -3.12789910e-03  1.00000000e+00]]
=> MatrixB =  [[-2.19189956e+00  1.08059785e-01  4.30714845e+02]
 [-2.98505403e+00 -4.48153295e-01  6.36091901e+02]
 [-7.19621378e-03  4.49285188e-04  1.00000000e+00]]
=> MatrixC =  [[-1.06122502e+00 -5.80395453e-02  3.81511526e+02]
 [-1.29407427e+00  2.14188277e-01  3.27761048e+02]
 [-3.18749314e-03 -1.48513790e-04  1.00000000e+00]]
=> MatrixD =  [[-6.56100043e-01 -2.23582382e+00  4.97126571e+02]
 [-4.64843971e-02 -2.77641520e+00  5.24733577e+02]
 [-1.21516486e-04 -7.20435663e-03  1.00000000e+00]]

=> pointXMin =  225
=> pointXMax =  411
=> pointYMin =  290
=> pointYMax =  519

==> 1. Size BEV
Height:  785
Width:  657

==> 2. Read Camera & Rotate

==> 2. Show  BEV Result

==> 3. Remove Small Square [BEST] Inside - [ F U L L ]

==> 3. Show Line to